## Read Khan Academy Subject Page

In [2]:
import os
from bs4 import BeautifulSoup
import requests
import pymongo
import re

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

service = Service(executable_path='r/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=service, options=options)

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

url = "https://www.khanacademy.org/math/probability/xa88397b6:display-quantitative/xa88397b6:frequency-tables-dot-plots/v/frequency-tables-and-dot-plots"
driver.get(url)


### Read Subject Page

In [4]:
# Read Khan Academy Subject Page
headers = {
   'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0',
   'X-Requested-With': 'XMLHttpRequest'
}
domain = 'https://www.khanacademy.org'
# source = domain + '/math/statistics-probability'
source = domain + '/math/probability'
response = requests.get(source, headers=headers)
response

# print(dir(response))

<Response [200]>

In [7]:
# Read Lesson Page using Selenium

# url = "https://www.khanacademy.org/math/probability/xa88397b6:display-quantitative/xa88397b6:frequency-tables-dot-plots/v/frequency-tables-and-dot-plots"
# url = "https://www.khanacademy.org/math/probability/xa88397b6:display-quantitative/xa88397b6:frequency-tables-dot-plots/e/creating-dot-plots"
url = "https://www.khanacademy.org/math/probability/xa88397b6:display-quantitative/xa88397b6:mean-median-data-displays/a/choosing-the-best-measure-of-center"
# driver.get(url)

# url = "https://freetts.com/"
# driver.get(url)

def getLessonMeta(url, type):

   driver.get(url)
   # https://selenium-python.readthedocs.io/waits.html

   # Wait till the page is fully loaded
   try:

      if type == 'Video':
         element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'span[data-test-id="video-time-hidden"]'))
         )

      elif type == 'Exercise':
         element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#content-library-content-footer-portal ._g20yn58 span'))
         )

      elif type == 'Article':
         element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.framework-perseus'))
         )

   except:
      return 'not found'

   # Get metadata

   # Get video length
   if type == 'Video':
      elements = driver.find_elements(By.CSS_SELECTOR, 'span[data-test-id="video-time-hidden"]')
      if len(elements) == 1:
         # Get # of seconds of the video
         x = re.search("[0-9]+:[0-9]+$", elements[0].text)
         video_length = x.group()
         min_seg = video_length.split(":")
         length = int(min_seg[0]) * 60 + int(min_seg[1])
      else:
         length = 0     # 'not found'
      return length

   # Get numbervideo length
   elif type == 'Exercise':
      elements = driver.find_elements(By.CSS_SELECTOR, '#content-library-content-footer-portal ._g20yn58 span')
      if len(elements) == 1:
         x = re.search("[0-9]+", elements[0].text)
         questions = int(x.group())
      else:
         questions = 0     # 'not found'
      return questions

   # Get article size in Kb
   elif type == 'Article':
      elements = driver.find_elements(By.CSS_SELECTOR, '.framework-perseus')
      if len(elements) == 1:
         size = len(elements[0].text) / 1024
      else:
         size = 0     # 'not found'
      return size


meta_data = getLessonMeta(url, 'Article')
# print('Exercise' + ' - ' + str(meta_data))
print(meta_data)


1.66796875


#### Read units

In [8]:
# Mount lessons array

# Lessons
lessons = []

# Get Page headings
unit_sequence = 0
# bsSubject = BeautifulSoup(response.text, 'lxml')
bsSubject = BeautifulSoup(response.text, 'html.parser')

field   = bsSubject.find('div', attrs={"aria-label": "breadcrumbs"}).get_text()
subject = bsSubject.find('h1',  attrs={"data-test-id": "unit-block-title"}).get_text()

# print(field)

for unit_block in bsSubject.findAll('div', class_='_12yy6f6l'):
   unit_sequence += 1
   unit_title = unit_block.find('h3').get_text()
   print(unit_title)

   for group in unit_block.findAll('a', attrs={"data-test-id": "lesson-link"}):
      group_title = group.get('title')
      group_link  = group.get('href')
      
      # Load group page
      group_page = requests.get(domain + group_link, headers=headers)
      print("\t" + group_title)
      bsGroup = BeautifulSoup(group_page.text, 'html.parser')
      unit_group = bsGroup.find('ul', attrs={"data-test-id": "learnable-content-cells"})

      # Browse through lessons
      for lesson in unit_group.findAll('li'):
         lesson_title = lesson.get_text()
         lesson_link = lesson.find('a').get('href')

         # Get lesson type
         lesson_type = lesson.find(['span', 'svg'], attrs={"role": "img"}).get('aria-label')
         if lesson_type not in ['Article', 'Video']:
            lesson_type = 'Exercise'

         # Load lesson metadata
         meta_data = getLessonMeta(domain + lesson_link, lesson_type)
         print("\t\t" + lesson_type + ' - ' + str(meta_data))




Unit 1: Displaying a single quantitative variable
	Frequency tables and dot plots
		Video - 439
		Exercise - 4
		Exercise - 4
	Histograms
		Video - 442
		Exercise - 4
		Video - 269
		Exercise - 4
	Mean and median in data displays
		Video - 534
		Video - 148
		Exercise - 4
		Video - 275
		Exercise - 4
		Article - 1.66796875
	Interquartile range
		Video - 372
		Exercise - 4
	Box and whisker plots
		Video - 198
		Video - not found
		Exercise - 4
		Video - 198


		Exercise - 4
		Video - 464


#### Read groups of exercises and save them in the database

In [5]:
# Read group of exercises

# Group of exercises
groups = []

# Get articles tab
group_sequence = 0
bs = BeautifulSoup(response.text, 'lxml')
for article in bs.findAll('div', {'class':'article-tab'}):

   for item in article.children:

      if str(item)[:3] == '<p>':    # Category titles

         group_item = {}
         category = item.text[:-2].strip()
         group_sequence += 100  

      elif str(item)[:3] == '<ul':  # Group items

         bs2 = BeautifulSoup(str(item), 'lxml')
         sequence = group_sequence

         for ex_page in bs2.findAll('a'):

            # Group properties
            raw_name = ex_page.text.split(' [ ')
            if len(raw_name) == 2:

               _id = raw_name[0]
               sequence += 1
               url = ex_page['href']
               url = url if url[:4] == 'http' else domain + url


               if _id != 'Python Challenges: Part -1':
                  quantity = int(''.join(c for c in raw_name[1] if c.isnumeric()))
               else:
                  quantity = 64

               group_item = {
                  '_id':      _id,
                  'category': category,
                  'sequence': sequence,
                  'url':      url,
                  'quantity': quantity,
                  'qtd_done': 0,
                  'header': ''
               }
               groups.append(group_item)

In [6]:
# print out an array or dictionary nicely
import json
print(json.dumps(groups, indent=4))

[
    {
        "_id": "Python Basic (Part -I)",
        "category": "List of Python Exercises",
        "sequence": 101,
        "url": "https://www.w3resource.com/python-exercises/python-basic-exercises.php",
        "quantity": 150,
        "qtd_done": 0,
        "header": ""
    },
    {
        "_id": "Python Basic (Part -II)",
        "category": "List of Python Exercises",
        "sequence": 102,
        "url": "https://www.w3resource.com/python-exercises/basic/",
        "quantity": 145,
        "qtd_done": 0,
        "header": ""
    },
    {
        "_id": "Python Programming Puzzles",
        "category": "List of Python Exercises",
        "sequence": 103,
        "url": "https://www.w3resource.com/python-exercises/puzzles/index.php",
        "quantity": 100,
        "qtd_done": 0,
        "header": ""
    },
    {
        "_id": "Python built-in Modules",
        "category": "List of Python Exercises",
        "sequence": 104,
        "url": "https://www.w3resource.com/pyt

In [ ]:
# Save group of the exercises in the database

# Create w3resource python database
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['khan-academy']

# group of exercises collection
colLessons = db["lessons"]

x = colLessons.delete_many({})
x = colLessons.insert_many(lessons)

#print list of the _id values of the inserted documents:
print(x.inserted_ids)